Analysing the outcomes of Teqball rallies in singles and doubles

Data Acquisition

In [252]:
## Importing essentials

import pandas as pd
import numpy as np

In [253]:
df_singles = pd.read_excel("All_Data.xlsx")

In [254]:
df_doubles = pd.read_excel("Doubles_All_Data.xlsx")

In [255]:
df_singles.head()

,Date,Country,Competition name,Competition type,Court Type,Category,Competition Stage,Set number,Rally number,Player A,Player B,Serving Player,Service,Point for Player…,Point type
0,2021-04-04,Cape Verde,Cape Verde Challenger Cup,Challenger Cup,Sand,Singles,Semi Final,1.0,1,Apor Gyorgydeak,Hugo Rabeux,B,1,B,UE
1,2021-04-04,Cape Verde,Cape Verde Challenger Cup,Challenger Cup,Sand,Singles,Semi Final,1.0,2,Apor Gyorgydeak,Hugo Rabeux,B,1,A,UE
2,2021-04-04,Cape Verde,Cape Verde Challenger Cup,Challenger Cup,Sand,Singles,Semi Final,1.0,3,Apor Gyorgydeak,Hugo Rabeux,B,1,B,W
3,2021-04-04,Cape Verde,Cape Verde Challenger Cup,Challenger Cup,Sand,Singles,Semi Final,1.0,4,Apor Gyorgydeak,Hugo Rabeux,B,2,A,UE
4,2021-04-04,Cape Verde,Cape Verde Challenger Cup,Challenger Cup,Sand,Singles,Semi Final,1.0,5,Apor Gyorgydeak,Hugo Rabeux,A,2,B,UE


In [256]:
df_doubles.head()

,Date,Country,Competition name,Competition type,Court Type,Category,Competition Stage,Team A,Player B,Serving Team,Service,Point for Team…,Point type,Unnamed: 13
0,2021-05-02,Hungary,Budapest Challenger Cup,Challenger Cup,Indoor,Doubles (open),Group Stage,Adrian Duszak / Franczuk Bartlomiej,Adam Toronyi / Patrik Szatmari,B,1,A,UE,NaN
1,2021-05-02,Hungary,Budapest Challenger Cup,Challenger Cup,Indoor,Doubles (open),Group Stage,Adrian Duszak / Franczuk Bartlomiej,Adam Toronyi / Patrik Szatmari,B,1,B,UE,NaN
2,2021-05-02,Hungary,Budapest Challenger Cup,Challenger Cup,Indoor,Doubles (open),Group Stage,Adrian Duszak / Franczuk Bartlomiej,Adam Toronyi / Patrik Szatmari,B,1,A,UE,NaN
3,2021-05-02,Hungary,Budapest Challenger Cup,Challenger Cup,Indoor,Doubles (open),Group Stage,Adrian Duszak / Franczuk Bartlomiej,Adam Toronyi / Patrik Szatmari,B,2,A,FE,NaN
4,2021-05-02,Hungary,Budapest Challenger Cup,Challenger Cup,Indoor,Doubles (open),Group Stage,Adrian Duszak / Franczuk Bartlomiej,Adam Toronyi / Patrik Szatmari,A,1,B,UE,NaN


Data Cleaning and Preprocessing

In [257]:
## Uniformize two datasets for common analysis

# Remove df_doubles unnecessary columns
df_doubles.drop(columns='Unnamed: 13', inplace= True)

# Remove Set number and rally number column from df_singles as it will be not needed for the analysis
df_singles.drop(columns=['Set number', 'Rally number'], inplace = True)

In [258]:
## Make the columns uniformal in both datasets
df_singles.columns

Index(['Date', 'Country', 'Competition name', 'Competition type', 'Court Type',
       'Category', 'Competition Stage', 'Player A', 'Player B',
       'Serving Player', 'Service', 'Point for Player…', 'Point type'],
      dtype='object')

In [259]:
df_doubles.columns

Index(['Date', 'Country', 'Competition name', 'Competition type', 'Court Type',
       'Category', 'Competition Stage', 'Team A', 'Player B', 'Serving Team',
       'Service', 'Point for Team…', 'Point type'],
      dtype='object')

In [260]:
df_singles.rename(columns={ 'Player A' : 'Team A',
                            'Player B' : 'Team B',
                            'Serving Player' : 'Serving Team',
                            'Point for Player…' : 'Point Winner'}, inplace = True)

In [261]:
df_doubles.rename(columns={'Player B' : 'Team B',
                           'Point for Team…' : 'Point Winner'}, inplace = True)

In [262]:
# Check if columns match
df_singles.columns == df_doubles.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [263]:
# Rename df_doubles 'Category' column values to from Doubles (open) to Doubles for simplicity
df_doubles['Category'] = 'Doubles'

In [264]:
# Merging the datasets
df = pd.concat([df_singles, df_doubles])

In [265]:
## Exploring the Dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3475 entries, 0 to 1427
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               3475 non-null   datetime64[ns]
 1   Country            3475 non-null   object        
 2   Competition name   3475 non-null   object        
 3   Competition type   3475 non-null   object        
 4   Court Type         3475 non-null   object        
 5   Category           3475 non-null   object        
 6   Competition Stage  3475 non-null   object        
 7   Team A             3475 non-null   object        
 8   Team B             3475 non-null   object        
 9   Serving Team       3475 non-null   object        
 10  Service            3475 non-null   object        
 11  Point Winner       3475 non-null   object        
 12  Point type         3475 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 380.1+ KB


In [266]:
# As all DF (Double Fault) services must be UEs (Unforced Error), make sure there are no mistakes like this
df['Point type'] = df.apply(lambda row: "UE" if (row['Service'] == "DF" and row['Point type'] != "UE") else row['Point type'], axis=1)

In [267]:
# Check unique values of Point type
df['Point type'].unique()

array(['UE', 'W', 'FE', 'E', 'w', 'Ue', 'ue', 'fe', 'e'], dtype=object)

In [268]:
df['Point type'] = df['Point type'].apply(lambda x : x.upper())

In [269]:
# Check names as it becomes important when analysing the players
unique = df['Team A'].unique().tolist()
for name in df['Team B']:
    if name not in unique:
        unique.append(name)

unique.sort()

In [272]:
# Reorder names in case of doubles teams in alphabetical order and remove empty space before and after names
def reorder(name):
    if name.split('/')[0].strip().lower() > name.split('/')[1].strip().lower():
        name = name.split('/')[1].strip() + ' / ' + name.split('/')[0].strip()
    return name


In [273]:
df['Team A'] = df['Team A'].apply(lambda x : reorder(x) if '/' in x else x.strip())
df['Team B'] = df['Team B'].apply(lambda x : reorder(x) if '/' in x else x.strip())